In [ ]:
# Import necessary libraries
!pip install -U tensorflow-addons
!pip install pydicom
!pip install transformers
!pip install folium==0.2.1
!pip install -q transformers datasetsimport pandas as pd
from tensorflow.keras.applications.densenet import DenseNet121
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, BatchNormalization, Conv2D, MaxPool2D
from tensorflow.keras.optimizers import Adam
from google.colab import drive
from PIL import Image
import os
from keras.models import Sequential
from keras.layers import Dense, Flatten, BatchNormalization, Dropout
from tensorflow.keras.optimizers import RMSprop
from sklearn.model_selection import KFold
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.applications.xception import Xception
import cv2
import pickle
import seaborn as sns
from sklearn.preprocessing import OneHotEncoder
from tqdm import tqdm
from sklearn.metrics import confusion_matrix
from keras.models import Model, load_model
from keras.layers import Input, Conv2D, MaxPool2D
from keras.preprocessing.image import ImageDataGenerator
import keras
import random
import shutil
import csv
import tensorflow as tf
import pydicom as dicom
import zipfile
from keras import optimizers
from keras.callbacks import ModelCheckpoint
from keras.applications.imagenet_utils import preprocess_input
from keras import backend as K
from keras.initializers import RandomNormal
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_addons as tfa
from torch.utils.data import DataLoader
from tqdm.notebook import tqdm
import torch
from transformers import ConvNextFeatureExtractor
from torchvision.transforms import Compose, Normalize, RandomHorizontalFlip, RandomResizedCrop, Resize, ToTensor
from transformers import AutoModelForImageClassification
from datasets import load_dataset
import statistics


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
drive.mount('/content/drive')

# Set the path to the datasets
train_path = '/content/drive/MyDrive/AML_Kaggle_Competition/train_images'
test_path = '/content/drive/MyDrive/AML_Kaggle_Competition/test_images'
labels_path = '/content/drive/MyDrive/AML_Kaggle_Competition/labels_train.csv'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


**1st MODEL: DENSENET121**

---



---



In [ ]:
# Load the CSV file containing the image labels
labels_df = pd.read_csv(labels_path)


In [ ]:
# Split the data into train and validation sets
train_df, val_df = train_test_split(labels_df, test_size=0.2, random_state=42, stratify=labels_df['class_id'])

train_df['class_id'] = train_df['class_id'].astype(str)


In [ ]:
# Define the ImageDataGenerator for data augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=10,
    zoom_range=0.2,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=False,
    fill_mode='nearest')

val_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
# Convert class_id values to strings in the validation data DataFrame
val_df['class_id'] = val_df['class_id'].astype(str)


In [ ]:
# Define target image size and hyperparameters
batch_size = 200
target_size = (224, 224)
epochs = 100
lr = 0.0001

In [ ]:
# Define the training and validation data generators
train_generator = train_datagen.flow_from_dataframe(
    train_df,
    directory=train_path,
    x_col='file_name',
    y_col='class_id',
    target_size=target_size,
    batch_size=batch_size,
    class_mode='categorical')

val_generator = val_datagen.flow_from_dataframe(
    val_df,
    directory=train_path,
    x_col='file_name',
    y_col='class_id',
    target_size=target_size,
    batch_size=batch_size,
    class_mode='categorical')


Found 4204 validated image filenames belonging to 3 classes.
Found 468 validated image filenames belonging to 3 classes.


In [ ]:
# Get a list of file paths for the test images
test_files = os.listdir(test_path)
test_files = [os.path.join(test_path, file) for file in test_files]

# Create a dataframe that lists the file paths
test_df = pd.DataFrame({'filename': test_files})

# Define the test data generator using flow_from_dataframe
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_dataframe(
    dataframe=test_df,
    x_col='filename',
    y_col=None,
    target_size=target_size,
    batch_size=batch_size,
    class_mode=None,
    shuffle=False
)


Found 1168 validated image filenames.


In [ ]:


# load pre-trained DenseNet121 model
base_model = DenseNet121(weights='imagenet', include_top=False, input_shape=(150, 150, 3))


# add custom layers on top
model = Sequential()
model.add(base_model)
model.add(BatchNormalization())
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(3, activation='softmax'))



In [ ]:
# freeze the weights of the pre-trained layers
for layer in base_model.layers:
    layer.trainable = False


In [ ]:

optimizer = RMSprop(learning_rate = lr)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
model.summary()


In [ ]:
# Define the checkpoint filepath and settings
checkpoint_filepath = '/content/drive/MyDrive/AML_Kaggle_Competition/AML_Coursework_DenseNet121_rmsprop.h5'
checkpoint_callback = ModelCheckpoint(
    checkpoint_filepath,
    monitor='val_accuracy',
    save_best_only=True,
    save_freq='epoch',  # Save after each epoch
    verbose=1
)

In [ ]:
from keras.models import load_model
model = load_model('/content/drive/MyDrive/AML_Kaggle_Competition/AML_Coursework_DenseNet121_rmsprop.h5')

In [ ]:
# Train the model
history = model.fit(train_generator, epochs=epochs, validation_data=val_generator, callbacks=[checkpoint_callback])


In [ ]:
# Make predictions on the test set
predictions = model.predict(test_generator)


In [ ]:
# Get the predicted class labels
predicted_classes = np.argmax(predictions, axis=1)

# Get the filenames of the test images and remove the path and directory
filenames = [filename.split('/')[-1] for filename in test_generator.filenames]

# Create a DataFrame containing the filenames and predicted classes
df1 = pd.DataFrame({'file_name': filenames, 'class_id': predicted_classes})

**2nd MODEL: XCEPTION**

---



---



In [ ]:
# Load the CSV file containing the image labels
labels_df = pd.read_csv(labels_path)


In [ ]:
# Define target image size and hyperparameters
batch_size = 32
target_size = (224, 224)
epochs = 100
lr = 0.0001
num_folds = 4

In [ ]:
# Define the KFold object
kf = KFold(n_splits=num_folds, shuffle=True, random_state=42)

# Define the checkpoint filepath and settings
checkpoint_filepath = '/content/drive/MyDrive/AML_Kaggle_Competition/AML_Coursework_Xception.h5'
checkpoint_callback = ModelCheckpoint(
    checkpoint_filepath,
    monitor='val_accuracy',
    save_best_only=True,
    save_freq='epoch',  # Save after each epoch
    verbose=1
)

In [ ]:
# Split the data into train and validation sets
train_df, val_df = train_test_split(labels_df, test_size=0.2, random_state=42, stratify=labels_df['class_id'])

train_df['class_id'] = train_df['class_id'].astype(str)


In [ ]:
# Define a function to normalize the pixel values
def normalize(x):
    x = x / 255.0
    mean = np.mean(x)
    std = np.std(x)
    return (x - mean) / std



In [ ]:

# Define the ImageDataGenerators
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=20,
    zoom_range=0.2,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=False,
    fill_mode='nearest')

val_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)


In [ ]:
# Define the training and validation data generators
train_generator = train_datagen.flow_from_dataframe(
    train_df,
    directory=train_path,
    x_col='file_name',
    y_col='class_id',
    target_size=target_size,
    batch_size=batch_size,
    class_mode='categorical')

val_generator = val_datagen.flow_from_dataframe(
    val_df,
    directory=train_path,
    x_col='file_name',
    y_col='class_id',
    target_size=target_size,
    batch_size=batch_size,
    class_mode='categorical')


In [ ]:
# Get a list of file paths for the test images
test_files = os.listdir(test_path)
test_files = [os.path.join(test_path, file) for file in test_files]

# Create a dataframe that lists the file paths
test_df = pd.DataFrame({'filename': test_files})

# Define the test data generator using flow_from_dataframe
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_dataframe(
    dataframe=test_df,
    x_col='filename',
    y_col=None,
    target_size=target_size,
    batch_size=batch_size,
    class_mode=None,
    shuffle=False
)


In [ ]:
# load pre-trained Xception model
base_model = Xception(weights='imagenet', include_top=False, input_shape=(224, 224, 3))


# add custom layers on top
model = Sequential()
model.add(base_model)
model.add(Flatten())
model.add(Dropout(0.3))
model.add(Dense(256, activation='relu'))
model.add(Dense(3, activation='softmax'))



In [ ]:
# freeze the weights of the pre-trained layers
for layer in base_model.layers:
    layer.trainable = False


In [ ]:

optimizer = Adam(learning_rate = lr)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
model.summary()


In [ ]:
# Define the learning rate reduction settings
learn_control = ReduceLROnPlateau(monitor='accuracy', patience=3, verbose=1, factor=.5, min_lr=0.000005)

In [ ]:
# Train the model
history = model.fit(train_generator, epochs=epochs, validation_data=val_generator, callbacks=[checkpoint_callback]) # learn_control


In [ ]:
# Make predictions on the test set
predictions = model.predict(test_generator)


In [ ]:
# Get the predicted class labels
predicted_classes = np.argmax(predictions, axis=1)

# Get the filenames of the test images
filenames = test_generator.filenames

# Create a DataFrame containing the filenames and predicted classes
df2 = pd.DataFrame({'file_name': filenames, 'class_id': predicted_classes})


**3d MODEL: VIT**

---



---



In [ ]:
drive.mount('/content/drive')

# preparation the data
num_classes = 3
input_shape = (256, 256, 3)
#Loading test images & test labels from the respective datasets
labels = pd.read_csv('/content/drive/MyDrive/AML_Kaggle_Competition/labels_train.csv')
norm_labels = []
norm_images = []
norm_path=os.listdir('/content/drive/MyDrive/AML_Kaggle_Competition/train_images')
# retrieving the lables to preserve the order
for image in tqdm(norm_path):
  lbl = labels.loc[labels.file_name == image, 'class_id'].iloc[-1]
  norm_labels.append(lbl)
  image = cv2.imread('/content/drive/MyDrive/AML_Kaggle_Competition/train_images/train' + image)
  image = cv2.resize(image, dsize=(256,256))
  norm_images.append(image)

# converting the images into numpy arays
norm_images = np.array(norm_images)
norm_labels = np.array(norm_labels)

In [ ]:
# train-test split (80%)
X_train = norm_images[:3737,:,:]
X_test=norm_images[3737:,:,:]
y_train = norm_labels[:3737]
y_test=norm_labels[3737:]

In [ ]:
# print the x_train & x_test shapes
print(f"x_train shape: {X_train.shape} - y_train shape: {y_train.shape}")
print(f"x_test shape: {X_test.shape} - y_test shape: {y_test.shape}")


In [ ]:
# adding one more axis to the test sets
y_train = y_train[:, np.newaxis]
y_test = y_test[:, np.newaxis]
# print with the new dimensions
print(f"x_train shape: {X_train.shape} - y_train shape: {y_train.shape}")
print(f"x_test shape: {X_test.shape} - y_test shape: {y_test.shape}")


In [ ]:
# configuration of the hyperparameters
learning_rate = 0.001
weight_decay = 0.0001
num_epochs = 100
batch_size = 256
image_size = 144  # We'll resize input images to this size
patch_size = 6  # Size of the patches to be extract from the input images
num_patches = (image_size // patch_size) ** 2
projection_dim = 64
num_heads = 4
transformer_units = [
    projection_dim * 2,
    projection_dim,
]
# Size of the transformer layers
transformer_layers = 8
mlp_head_units = [2048, 1024]  # Size of the dense layers of the final classifier

In [ ]:
# image augmentation
data_augmentation = keras.Sequential(
    [
        layers.Normalization(),
        layers.Resizing(image_size, image_size),
        layers.RandomFlip("horizontal"),
        layers.RandomRotation(factor=0.02),
        layers.RandomZoom(
            height_factor=0.2, width_factor=0.2
        ),
    ],
    name="data_augmentation",
)
# Compute the mean and the variance of the training data for normalization.
data_augmentation.layers[0].adapt(X_train)

In [ ]:
# function to implement multilayer perceptron (MLP)
def mlp(x, hidden_units, dropout_rate):
    for units in hidden_units:
        x = layers.Dense(units, activation=tf.nn.gelu)(x)
        x = layers.Dropout(dropout_rate)(x)
    return x

In [ ]:
# class to implement patch creation as a layer
class Patches(layers.Layer):
    def __init__(self, patch_size):
        super(Patches, self).__init__()
        self.patch_size = patch_size

    def call(self, images):
        batch_size = tf.shape(images)[0]
        patches = tf.image.extract_patches(
            images=images,
            sizes=[1, self.patch_size, self.patch_size, 1],
            strides=[1, self.patch_size, self.patch_size, 1],
            rates=[1, 1, 1, 1],
            padding="VALID",
        )
        patch_dims = patches.shape[-1]
        patches = tf.reshape(patches, [batch_size, -1, patch_dims])
        return patches

In [ ]:
# implementation of the patch encoding layer
class PatchEncoder(layers.Layer):
    def __init__(self, num_patches, projection_dim):
        super(PatchEncoder, self).__init__()
        self.num_patches = num_patches
        self.projection = layers.Dense(units=projection_dim)
        self.position_embedding = layers.Embedding(
            input_dim=num_patches, output_dim=projection_dim
        )

    def call(self, patch):
        positions = tf.range(start=0, limit=self.num_patches, delta=1)
        encoded = self.projection(patch) + self.position_embedding(positions)
        return encoded

In [ ]:
# built the ViT model
def create_vit_classifier():
    inputs = layers.Input(shape=input_shape)
    # Augment data.
    augmented = data_augmentation(inputs)
    # Create patches.
    patches = Patches(patch_size)(augmented)
    # Encode patches.
    encoded_patches = PatchEncoder(num_patches, projection_dim)(patches)

    # Create multiple layers of the Transformer block.
    for _ in range(transformer_layers):
        # Layer normalization 1.
        x1 = layers.LayerNormalization(epsilon=1e-6)(encoded_patches)
        # Create a multi-head attention layer.
        attention_output = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=projection_dim, dropout=0.1
        )(x1, x1)
        # Skip connection 1.
        x2 = layers.Add()([attention_output, encoded_patches])
        # Layer normalization 2.
        x3 = layers.LayerNormalization(epsilon=1e-6)(x2)
        # MLP.
        x3 = mlp(x3, hidden_units=transformer_units, dropout_rate=0.1)
        # Skip connection 2.
        encoded_patches = layers.Add()([x3, x2])

    # Create a [batch_size, projection_dim] tensor.
    representation = layers.LayerNormalization(epsilon=1e-6)(encoded_patches)
    representation = layers.Flatten()(representation)
    representation = layers.Dropout(0.5)(representation)
    # Add MLP.
    features = mlp(representation, hidden_units=mlp_head_units, dropout_rate=0.5)
    # Classify outputs.
    logits = layers.Dense(num_classes)(features)
    # Create the Keras model.
    model = keras.Model(inputs=inputs, outputs=logits)
    return model


In [ ]:
# compile, train, and evaluate the mode
def run_experiment(model):
    optimizer = tfa.optimizers.AdamW(learning_rate=learning_rate, weight_decay=weight_decay)

    model.compile(optimizer=optimizer, loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=[keras.metrics.SparseCategoricalAccuracy(name="accuracy"),keras.metrics.SparseTopKCategoricalAccuracy(5, name="top-5-accuracy")])

    checkpoint_filepath = "/content/drive/MyDrive/AML_Kaggle_Competition/vit.h5"
    checkpoint_callback = keras.callbacks.ModelCheckpoint(
        checkpoint_filepath,
        monitor="val_accuracy",
        save_best_only=True,
        save_weights_only=True)

    history = model.fit(
        x=X_train,
        y=y_train,
        epochs= num_epochs,
        validation_data=(X_test, y_test),
        callbacks=[checkpoint_callback ]
    )

    model.load_weights(checkpoint_filepath)

    return history


vit_classifier = create_vit_classifier()
history = run_experiment(vit_classifier)

In [ ]:
# plot accuracy scores and Loss value
plt.figure(figsize=(8,6))
plt.title('Accuracy scores')
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.legend(['acc', 'val_acc'])
plt.show()
plt.figure(figsize=(8,6))
plt.title('Loss value')
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.legend(['loss', 'val_loss'])
plt.show()

In [ ]:
# loading target images from the respective dataset
target_images = []
file_name=[]
test_path=os.listdir('/content/drive/MyDrive/AML_Kaggle_Competition/test_images')
for test_image in tqdm(test_path):
  file_name.append(test_image)
  test_image = cv2.imread('/content/drive/MyDrive/AML_Kaggle_Competition/test_images/' + test_image)
  test_image = cv2.resize(test_image, dsize=(256,256))
  target_images.append(test_image)

target_images = np.array(target_images)
Target = target_images


In [ ]:
# prediction of test images
predictions = vit_classifier.predict(Target)
predictions = predictions.argmax(axis=1)

In [ ]:
# creation of the 1st prediction dataframe
df3 = pd.DataFrame()
df3['file_name'] = file_name
df3['class_id'] = predictions

**MAJORITY VOTE OF THE THREE MODEL RESULTS**

---



---



In [ ]:
# Merge the three DataFrames on the 'file_name' column
merged_df = df1.merge(df2, on='file_name').merge(df3, on='file_name')

# Compute the mode (most frequent value) of the 'class_id' columns for each row
mode_series = merged_df[['class_id_x', 'class_id_y', 'class_id']].mode(axis=1)

# Extract the mode values and convert them to integers
mode_values = mode_series.iloc[:, 0].astype(int)

# Create a new DataFrame with the file names and mode values
results_df = pd.DataFrame({'file_name': merged_df['file_name'], 'class_id': mode_values})

# Save the DataFrame to a CSV file
results_df.to_csv('/content/drive/MyDrive/AML_results/resultstobesent_majority.csv', index=False)